In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import pandas as pd
from IPython.display import JSON
from datetime import datetime
from dotenv import load_dotenv

# Set option to display all columns
pd.set_option('display.max_columns', None)

/var/folders/hl/5yxgmlvs0rxfpy8dgdyqq3740000gn/T/ipykernel_6166/1779869941.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [22]:
# Authentication
load_dotenv()
client_id = os.getenv('SPOTIPY_CLIENT_ID')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, 
                     requests_timeout=10, retries=5, status_retries=5, backoff_factor=0.5)

In [23]:
track_id = '0WNZvcLz94HQffsAw2OsYi'

#track = sp.track(track_id)
#album = sp.album(track['album']['id'])
#artist = sp.artist(track['artists'][0]['id'])
audio_features = sp.audio_features(track_id)[0]
#audio_analysis = sp.audio_analysis(track_id)

In [24]:
JSON(audio_features)

<IPython.core.display.JSON object>

In [27]:
JSON(sp.audio_analysis(track_id))

<IPython.core.display.JSON object>

In [28]:
def get_track_info(track_id):

    # call API to retrieve information on track ID
    track = sp.track(track_id)
    album = sp.album(track['album']['id'])
    artist = sp.artist(track['artists'][0]['id'])
    audio_features = sp.audio_features(track_id)[0]
    audio_analysis = sp.audio_analysis(track_id)
    
    track_info = {
        'track_id': track['name'],
        'track_name': track['id'],
        'duration_ms': track['duration_ms'],
        'track_popularity': track['popularity'],
        'explicit': track['explicit'],
        'album_name': album['name'],
        'album_id': album['id'],
        'album_type': album['album_type'],
        'album_total_tracks': album['total_tracks'],
        'release_date': album['release_date'],
        'artist_name': artist['name'],
        'artist_id': artist['id'],
        'artist_genres': '; '.join(artist['genres']),
        'artist_popularity': artist['popularity'],
        'artist_followers': artist['followers']['total'],
        'danceability': audio_features['danceability'],
        'energy': audio_features['energy'],
        'key': audio_features['key'],
        'key_confidence': audio_analysis['track']['key_confidence'],
        'loudness': audio_features['loudness'],
        'mode': audio_features['mode'],
        'mode_confidence': audio_analysis['track']['mode_confidence'],
        'speechiness': audio_features['speechiness'],
        'acousticness': audio_features['acousticness'],
        'instrumentalness': audio_features['instrumentalness'],
        'liveness': audio_features['liveness'],
        'valence': audio_features['valence'],
        'tempo': audio_features['tempo'],
        'tempo_confidence': audio_analysis['track']['tempo_confidence'],
        'time_signature': audio_features['time_signature'],
        'time_signature_confidence': audio_analysis['track']['time_signature_confidence']}

    return pd.DataFrame([track_info])

In [29]:
track_id = '0WNZvcLz94HQffsAw2OsYi'
get_track_info(track_id)

,track_id,track_name,duration_ms,track_popularity,explicit,album_name,album_id,album_type,album_total_tracks,release_date,artist_name,artist_id,artist_genres,artist_popularity,artist_followers,danceability,energy,key,key_confidence,loudness,mode,mode_confidence,speechiness,acousticness,instrumentalness,liveness,valence,tempo,tempo_confidence,time_signature,time_signature_confidence
0,The Golden Age - Bay Ledges Remix,0WNZvcLz94HQffsAw2OsYi,181363,37,False,Utopia Remixed,5uosr2ptUdWv7f1FOmmK3x,album,8,2023-07-14,St. Lucia,5WId4o5jdGVhptNU0uqKxu,indietronica; metropopolis; neo-synthpop; nyc ...,45,241346,0.584,0.649,4,0.788,-8.463,0,0.623,0.0307,0.00304,0.412,0.199,0.446,89.023,0.47,4,0.968


In [63]:
def get_track_ids_from_playlist(playlist_id, batch_size=100, top_n=0):

    # get total count of tracks in playlist
    total_tracks = sp.playlist_tracks(playlist_id)['total']

    if top_n > 0:
        total_tracks = min(total_tracks, top_n)
    
    # Fetch tracks from the playlist
    track_ids = []
    
    for offset in range(0, total_tracks, batch_size):
        tracks = sp.playlist_tracks(playlist_id, limit=batch_size, offset=offset)['items']
    
        for item in tracks:
            track = item['track']
            if track is not None:
                track_ids.append(track['id'])

    return track_ids

In [74]:
def get_track_info_for_playlist(playlist_id, batch_size=100, top_n=0, playlist_name=None):
    
    # get all track ids for playlist
    playlist_track_ids = get_track_ids_from_playlist(playlist_id, batch_size=batch_size, top_n=top_n)
    cnt_tracks_ids = len(playlist_track_ids)

    # get playlist name (if not specified by user)
    if playlist_name is None:
        playlist_name = sp.playlist(playlist_id)['name']

    # initialize playlist dataset
    playlist_tracks = pd.DataFrame()

    for start in range(0, cnt_tracks_ids, batch_size):
    
        # loop through each track ids in the playlist
        for track_id in playlist_track_ids[start:(start+batch_size)]:
            # get track id information
            track_info = get_track_info(track_id)
        
            # add track information to playlist dataset
            playlist_tracks = pd.concat([playlist_tracks, track_info], ignore_index=True)


    playlist_tracks['playlist_name'] = playlist_name

    # user message and output
    print(f"\nThe playlist '{playlist_name}' has {cnt_tracks_ids} tracks:\n")
    display(playlist_tracks.head())
    
    return playlist_tracks

In [80]:
def save_playlist(playlist_id, playlist_data=None, batch_size=100, top_n=0, playlist_name=None):

    # get playlist name (if not specified by user)
    if playlist_name is None:
        playlist_name = sp.playlist(playlist_id)['name']
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = f'playlist_data/{playlist_name}_{timestamp}.csv'

    # if playlist_data not passed as parameter, get track info through API
    if playlist_data is None:
        playlist_data = get_track_info_for_playlist(playlist_id, batch_size=batch_size, top_n=top_n, playlist_name=playlist_name)

    # save to csv
    playlist_data.to_csv(filepath, index=False)

    # user message and output
    print(f"\nThe playlist '{playlist_name}' has been saved at: '{filepath}'.\n")

    return playlist_data

In [68]:
# playlist ids
#playlist_id = '0336q5NsTO1sRYE68BpkGb' # liked_songs
playlist_id = '37i9dQZEVXcNQ1CcSphwMX' # discover weekly

In [66]:
playlist_df = get_track_info_for_playlist(playlist_id, top_n=50, batch_size=10)


The playlist 'Discover Weekly' has 30 tracks:



,track_id,track_name,duration_ms,track_popularity,explicit,album_name,album_id,album_type,album_total_tracks,release_date,artist_name,artist_id,artist_genres,artist_popularity,artist_followers,danceability,energy,key,key_confidence,loudness,mode,mode_confidence,speechiness,acousticness,instrumentalness,liveness,valence,tempo,tempo_confidence,time_signature,time_signature_confidence,playlist_name
0,Lights Go Down,6Q7fxyIXgrbk1bIySqbaij,183460,56,False,LIFTED,6jR7uovSlToKvLXjTrPZSV,album,9,2022-04-08,Hayden James,4csQIMQm6vI2A2SCVDuM2z,aussietronica; gauze pop; house,58,253288,0.893,0.820,3,0.507,-3.958,1,0.348,0.0578,0.002870,0.792000,0.0510,0.7810,123.005,0.836,4,1.000,Discover Weekly
1,I Can't Help But Feel,4zgGNKt9YQGUQHRu9FgZPM,178780,64,False,Hidden Youth,0p6lFaFEpTcukkXu793jXB,album,14,2022-08-26,Surfaces,4ETSs924pXMzjIeD6E9b4u,bedroom soul,64,912546,0.722,0.705,1,0.387,-4.750,0,0.419,0.0324,0.302000,0.000000,0.1230,0.7150,115.005,0.596,4,0.997,Discover Weekly
2,Slave To Your Love,6XS4mIgpD24d9MMGwLMPvf,152900,35,False,Slave To Your Love,4gc0pu0KQg1UCDrcn8n43s,single,1,2022-02-21,Soneland,628R8bjTnqSiTwQV45t4nY,,15,50,0.732,0.757,10,0.355,-8.879,0,0.351,0.0467,0.236000,0.883000,0.0781,0.2030,115.985,0.933,4,1.000,Discover Weekly
3,Control,1C26HwnwtIwLlSHJdB2iZw,187364,48,False,Control,3o3OAEZXFVzDavifKXa1aV,single,1,2020-07-10,Moods,14uVJsPC4DByeuD0cq36ez,chillhop; indie soul,46,46354,0.704,0.470,7,0.661,-6.890,0,0.744,0.3120,0.057000,0.000007,0.0891,0.4860,59.209,0.156,4,1.000,Discover Weekly
4,Spiral - jackLNDN Remix,6DwqtA14iC78eXxjD1sDOo,167500,44,False,Spiral (jackLNDN Remix),3fXZyb9K4Py1zRwdyjWeaL,single,2,2022-12-02,Lena Leon,0izKfMblL8LX6Bv2wG3Cy7,,41,2981,0.620,0.678,7,0.569,-9.334,0,0.399,0.0336,0.000536,0.725000,0.1220,0.0398,122.990,0.700,4,1.000,Discover Weekly


In [67]:
save_playlist(playlist_id, playlist_data=playlist_df, batch_size=50)


The playlist 'Discover Weekly' has been saved at: 'playlist_data/Discover Weekly_20240129_160646.csv'.



In [81]:
country_playlist_id = '37i9dQZF1DWTkxQvqMy4WW'
house_playlist_id = '37i9dQZF1DXa8NOEUWPn9W'
rap_playlist_id = '37i9dQZF1DX0XUsuxWHRQd'

country_df = save_playlist(country_playlist_id, top_n=50, batch_size=10, playlist_name='Country')
house_df = save_playlist(house_playlist_id, top_n=50, batch_size=10, playlist_name='House')
rap_df = save_playlist(rap_playlist_id, top_n=50, batch_size=10, playlist_name='Rap')


The playlist 'Country' has 50 tracks:



,track_id,track_name,duration_ms,track_popularity,explicit,album_name,album_id,album_type,album_total_tracks,release_date,artist_name,artist_id,artist_genres,artist_popularity,artist_followers,danceability,energy,key,key_confidence,loudness,mode,mode_confidence,speechiness,acousticness,instrumentalness,liveness,valence,tempo,tempo_confidence,time_signature,time_signature_confidence,playlist_name
0,Something in the Orange,3WMj8moIAXJhHsyLaqIIHI,228013,91,False,Something in the Orange,1CmTOKCeyz1aHH04OwvTPv,single,4,2022-04-22,Zach Bryan,40ZNYROS4zLfyyBSs2PGe2,classic oklahoma country,87,3128165,0.369,0.192,4,0.230,-12.151,0,0.459,0.0400,0.5550,0.000008,0.0954,0.148,175.212,0.113,3,1.000,Country
1,Fast Car,1Lo0QY9cvc8sUB2vnIOxDT,265493,88,False,Gettin' Old,5Uly85dJHHDfHQCsyUQ8gw,album,18,2023-03-24,Luke Combs,718COspgdWOnwOFpJHRZHS,contemporary country; country,82,9913044,0.712,0.603,8,0.327,-5.520,1,0.479,0.0262,0.1860,0.000000,0.1150,0.670,97.994,0.176,4,1.000,Country
2,Die From A Broken Heart,1kFpL1itkwkdhO6kcmNJra,188290,65,False,Die From A Broken Heart,4DPIFHZfh65TPPGem973zy,single,1,2018-10-19,Maddie & Tae,34bhyY8jfKez7uKakMfy4y,contemporary country; country; country road,61,1138829,0.607,0.514,4,0.123,-6.631,1,0.334,0.0323,0.3630,0.000000,0.2580,0.414,75.978,0.111,4,0.996,Country
3,Tennessee Whiskey,3fqwjXwUGN6vbzIwvyFMhx,293293,86,False,Traveller,7lxHnls3yQNl8B9bILmHj7,album,14,2015-05-04,Chris Stapleton,4YLtscXsxbVgi031ovDDdh,contemporary country; outlaw country,80,6576591,0.392,0.370,9,0.387,-10.888,1,0.498,0.0298,0.2050,0.009600,0.0821,0.512,48.718,0.015,4,1.000,Country
4,Automatic,0B0y2FImat9j9MJTNRZWfW,247826,63,False,Platinum,4ZnIw6llQFlXcFgjMNtrTw,album,16,2014-06-02,Miranda Lambert,66lH4jAE7pqPlOlzUKbwA0,contemporary country; country; country dawn; c...,67,3559622,0.591,0.703,10,1.000,-6.211,1,0.762,0.0256,0.0435,0.000002,0.1450,0.402,95.993,0.660,4,0.687,Country



The playlist 'Country' has been saved at: 'playlist_data/Country_20240129_163739.csv'.


The playlist 'House' has 50 tracks:



,track_id,track_name,duration_ms,track_popularity,explicit,album_name,album_id,album_type,album_total_tracks,release_date,artist_name,artist_id,artist_genres,artist_popularity,artist_followers,danceability,energy,key,key_confidence,loudness,mode,mode_confidence,speechiness,acousticness,instrumentalness,liveness,valence,tempo,tempo_confidence,time_signature,time_signature_confidence,playlist_name
0,Stay High,7LXimIqTYO76Utly8VFABu,188320,68,False,Stay High,3E3JRB0gGzHuc56GB3hUPJ,single,2,2023-11-24,Diplo,5fMUXHkw8R8eOP2RNVYEZX,dance pop; edm; electro house; moombahton; pop...,76,2579376,0.712,0.820,4,0.846,-6.624,0,0.692,0.0382,0.002260,0.3590,0.1000,0.322,128.005,0.917,4,0.995,House
1,Hella Good,1ck6fXDzbKiWKCVSfqxsUB,175757,57,False,Hella Good,7r5MlpWk6gQ2fCRDWjUIzn,single,1,2023-10-06,Niko The Kid,0FPoqGEZFwHQfu5tRPL08X,tech house,40,13213,0.703,0.930,8,0.058,-6.410,1,0.275,0.0589,0.000427,0.0951,0.1400,0.597,124.993,1.000,4,1.000,House
2,Frikitona,5ls4FQAoSlAFPFnaYpeyhn,174375,69,False,Frikitona,5LdsLkZqLq2oUEthzHiR9g,single,1,2023-07-14,Andruss,6HZwb7Zbnvfo8u1sst4QrI,mexican tech house; tech house,51,18249,0.787,0.964,6,0.000,-6.340,0,0.058,0.0639,0.083800,0.1760,0.0465,0.719,127.991,0.775,4,1.000,House
3,Carry Me Higher - 7 Inch Version,7tm4pF2JvKc4GGKSagRuS9,270402,65,False,Carry Me Higher,21q5qrGNBSI1ihI0TT5348,single,5,2023-11-17,The Blessed Madonna,4TvhRzxIL1le2PWCeUqxQw,edm,64,64056,0.850,0.537,6,0.516,-7.840,0,0.676,0.0519,0.003470,0.0839,0.1110,0.295,126.968,0.778,4,1.000,House
4,Smoke In Your Eyes,2Kurh5hEYRgzDzuahjHytQ,391503,60,False,Smoke In Your Eyes,5EEitGHmxCB0XD0rtV3bE7,single,1,2023-12-22,Skepta,2p1fiYHYiXz9qi0JJyxBzN,birmingham grime; grime; instrumental grime; u...,73,1214200,0.791,0.738,1,0.838,-9.792,1,0.605,0.0426,0.001390,0.8210,0.0861,0.412,128.003,0.779,4,1.000,House



The playlist 'House' has been saved at: 'playlist_data/House_20240129_163829.csv'.


The playlist 'Rap' has 50 tracks:



,track_id,track_name,duration_ms,track_popularity,explicit,album_name,album_id,album_type,album_total_tracks,release_date,artist_name,artist_id,artist_genres,artist_popularity,artist_followers,danceability,energy,key,key_confidence,loudness,mode,mode_confidence,speechiness,acousticness,instrumentalness,liveness,valence,tempo,tempo_confidence,time_signature,time_signature_confidence,playlist_name
0,redrum,52eIcoLUM25zbQupAZYoFh,270697,94,True,american dream,2RRYaYHY7fIIdvFlvgb5vq,album,15,2024-01-12,21 Savage,1URnnhqYAYcrqrcwql10ft,atl hip hop; hip hop; rap,93,16803819,0.624,0.740,2,0.548,-8.445,1,0.556,0.0481,0.00529,0.000224,0.500,0.246,172.089,0.147,4,0.981,Rap
1,First Person Shooter (feat. J. Cole),7aqfrAY2p9BUSiupwk3svU,247444,90,True,For All The Dogs,4czdORdCWP9umpbhFXK2fW,album,23,2023-10-06,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop; canadian pop; hip hop; pop r...,96,84302378,0.470,0.640,2,0.118,-7.779,1,0.158,0.3200,0.02770,0.000000,0.382,0.248,163.920,0.182,4,1.000,Rap
2,Surround Sound (feat. 21 Savage & Baby Tate),1udwFobQ1JoOdWPQrp2b6u,229938,94,True,The Forever Story (Extended Version),4rJDCELWL0fjdmN9Gn4f4g,album,16,2022-10-31,JID,6U3ybJ9UHNKEdsH7ktGBZ7,hip hop; pop rap; rap; underground hip hop,79,1566857,0.575,0.560,5,0.000,-7.302,1,0.072,0.4780,0.17300,0.000000,0.257,0.471,75.956,0.068,4,0.966,Rap
3,Let's Go,5GCU2LJWPzoP8Q6nexs61z,140106,79,True,Glockoma 2 (Deluxe),49vCWZ0yKkRvfetjsYVLnx,album,23,2023-06-23,Key Glock,0RESbWvOMyua0yuyVrztJ5,memphis hip hop; rap; southern hip hop; tennes...,77,1869826,0.673,0.526,8,0.814,-7.612,1,0.771,0.0677,0.00172,0.000000,0.300,0.303,75.505,0.090,4,1.000,Rap
4,née-nah,2yUzr8Sr6ldG8vmHhZwTnz,220584,89,True,american dream,2RRYaYHY7fIIdvFlvgb5vq,album,15,2024-01-12,21 Savage,1URnnhqYAYcrqrcwql10ft,atl hip hop; hip hop; rap,93,16803819,0.678,0.758,1,0.367,-6.524,1,0.499,0.2270,0.00495,0.000000,0.570,0.318,165.423,0.432,4,1.000,Rap



The playlist 'Rap' has been saved at: 'playlist_data/Rap_20240129_163921.csv'.



In [82]:
three_genre_df = pd.concat([country_df, house_df, rap_df])
three_genre_df.reset_index(drop=True, inplace=True)

In [83]:
three_genre_df.to_csv('playlist_data/50_country_house_rap.csv', index=False)

In [86]:
three_genre_df.head()

,track_id,track_name,duration_ms,track_popularity,explicit,album_name,album_id,album_type,album_total_tracks,release_date,artist_name,artist_id,artist_genres,artist_popularity,artist_followers,danceability,energy,key,key_confidence,loudness,mode,mode_confidence,speechiness,acousticness,instrumentalness,liveness,valence,tempo,tempo_confidence,time_signature,time_signature_confidence,playlist_name
0,Something in the Orange,3WMj8moIAXJhHsyLaqIIHI,228013,91,False,Something in the Orange,1CmTOKCeyz1aHH04OwvTPv,single,4,2022-04-22,Zach Bryan,40ZNYROS4zLfyyBSs2PGe2,classic oklahoma country,87,3128165,0.369,0.192,4,0.230,-12.151,0,0.459,0.0400,0.5550,0.000008,0.0954,0.148,175.212,0.113,3,1.000,Country
1,Fast Car,1Lo0QY9cvc8sUB2vnIOxDT,265493,88,False,Gettin' Old,5Uly85dJHHDfHQCsyUQ8gw,album,18,2023-03-24,Luke Combs,718COspgdWOnwOFpJHRZHS,contemporary country; country,82,9913044,0.712,0.603,8,0.327,-5.520,1,0.479,0.0262,0.1860,0.000000,0.1150,0.670,97.994,0.176,4,1.000,Country
2,Die From A Broken Heart,1kFpL1itkwkdhO6kcmNJra,188290,65,False,Die From A Broken Heart,4DPIFHZfh65TPPGem973zy,single,1,2018-10-19,Maddie & Tae,34bhyY8jfKez7uKakMfy4y,contemporary country; country; country road,61,1138829,0.607,0.514,4,0.123,-6.631,1,0.334,0.0323,0.3630,0.000000,0.2580,0.414,75.978,0.111,4,0.996,Country
3,Tennessee Whiskey,3fqwjXwUGN6vbzIwvyFMhx,293293,86,False,Traveller,7lxHnls3yQNl8B9bILmHj7,album,14,2015-05-04,Chris Stapleton,4YLtscXsxbVgi031ovDDdh,contemporary country; outlaw country,80,6576591,0.392,0.370,9,0.387,-10.888,1,0.498,0.0298,0.2050,0.009600,0.0821,0.512,48.718,0.015,4,1.000,Country
4,Automatic,0B0y2FImat9j9MJTNRZWfW,247826,63,False,Platinum,4ZnIw6llQFlXcFgjMNtrTw,album,16,2014-06-02,Miranda Lambert,66lH4jAE7pqPlOlzUKbwA0,contemporary country; country; country dawn; c...,67,3559622,0.591,0.703,10,1.000,-6.211,1,0.762,0.0256,0.0435,0.000002,0.1450,0.402,95.993,0.660,4,0.687,Country
